In [1]:
%pip install openai httpx==0.27.2

StatementMeta(, 42d04099-c385-476d-985d-ab490a267590, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 26.1 MB/s eta 0:00:00


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-b3416129-5ba8-460b-8cba-663a46038b3b
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fsspec-wrapper 0.1.14 requires PyJWT>=2.6.0, but you have pyjwt 2.4.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U openai

StatementMeta(, 42d04099-c385-476d-985d-ab490a267590, 13, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install phidata python-dotenv fastapi uvicorn groq httpx


StatementMeta(, 42d04099-c385-476d-985d-ab490a267590, 19, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.9/716.9 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 17.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

os.environ["GROQ_API_KEY"] = ""
os.environ["PHI_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""


StatementMeta(, 42d04099-c385-476d-985d-ab490a267590, 22, Finished, Available, Finished)

In [ ]:
from phi.agent import Agent
from phi.model.groq import Groq
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, min, max, stddev, hour, dayofweek, month
from pyspark.sql.types import IntegerType, DoubleType, DateType, TimestampType, StringType
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64

class EnhancedDataAnalysisAgent:
    def __init__(self, spark_session, table_name, limit=1000):
        self.spark = spark_session
        self.table_name = table_name
        self.agent = Agent(
            name="Enhanced Data Analysis Agent",
            model=Groq(id="llama3-70b-8192"),
            instructions=[
                "You are a sophisticated data analysis expert with domain knowledge across various industries.",
                "Generate both business and technical descriptions of datasets.",
                "Identify patterns, trends, anomalies, and insights specific to the domain of the data.",
                "Provide detailed data quality reports with comprehensive recommendations.",
                "Generate executable PySpark code to clean, transform, and analyze the data.",
                "Suggest business-relevant visualizations tailored to the dataset's domain.",
                "Provide actionable business insights based on data patterns."
            ],
            markdown=True,
        )
        
        # Load the dataset sample
        self.df = spark_session.sql(f"SELECT * FROM {table_name} LIMIT {limit}")
        self.df.createOrReplaceTempView("dataset_sample")
        
        # Set up logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    def generate_dataset_description(self):
        """
        Generate comprehensive business and technical descriptions of the dataset.
        Returns:
            dict: A dictionary containing business and technical descriptions.
        """
        try:
            # Analyze schema
            schema = self.df.schema
            column_names = [field.name for field in schema]
            column_types = {field.name: str(field.dataType) for field in schema}
            self.logger.info(f"Dataset columns: {column_names}")
            
            # Get basic statistics for each column
            self.logger.info("Calculating column statistics...")
            stats = {}
            
            for col_name in column_names:
                col_type = column_types[col_name]
                
                # Numeric statistics
                if any(type_name in col_type for type_name in ["Integer", "Double", "Float"]):
                    stats[col_name] = {
                        "type": col_type,
                        "min": self.df.agg({col_name: "min"}).collect()[0][0],
                        "max": self.df.agg({col_name: "max"}).collect()[0][0],
                        "avg": self.df.agg({col_name: "avg"}).collect()[0][0],
                        "stddev": self.df.agg({col_name: "stddev"}).collect()[0][0],
                        "null_count": self.df.filter(col(col_name).isNull()).count(),
                        "unique_values": self.df.select(col_name).distinct().count()
                    }
                
                # String statistics
                elif "String" in col_type:
                    top_values = self.df.groupBy(col_name).count().orderBy(col("count").desc()).limit(5)
                    stats[col_name] = {
                        "type": col_type,
                        "null_count": self.df.filter(col(col_name).isNull()).count(),
                        "unique_values": self.df.select(col_name).distinct().count(),
                        "top_values": top_values.collect()
                    }
                
                # Date/Timestamp statistics
                elif any(type_name in col_type for type_name in ["Date", "Timestamp"]):
                    stats[col_name] = {
                        "type": col_type,
                        "min": self.df.agg({col_name: "min"}).collect()[0][0],
                        "max": self.df.agg({col_name: "max"}).collect()[0][0],
                        "null_count": self.df.filter(col(col_name).isNull()).count()
                    }
                
                # Other types
                else:
                    stats[col_name] = {
                        "type": col_type,
                        "null_count": self.df.filter(col(col_name).isNull()).count(),
                        "unique_values": self.df.select(col_name).distinct().count()
                    }
            
            # Sample data
            sample_data = self.df.limit(10).toPandas().to_string()
            
            # Generate the agent prompt for dataset description
            description_prompt = f"""
            I need a comprehensive description of a dataset with the following characteristics:
            
            Schema:
            {schema}
            
            Column Statistics:
            {stats}
            
            Sample Data:
            {sample_data}
            
            Please provide:
            
            1. BUSINESS DESCRIPTION:
               - What domain/industry does this data likely belong to?
               - What business processes or activities does this data represent?
               - Who would typically use this data and for what purposes?
               - What are the key entities and relationships in this dataset?
            
            2. TECHNICAL DESCRIPTION:
               - Detailed description of each column (what it represents, data quality, value ranges)
               - Data structure and relationships
               - Potential primary keys and foreign keys
               - Data quality assessment
               - Potential transformations needed
            
            Be specific and provide details based on the actual values and patterns in the data, not just the schema.
            """
            
            # Use the agent to generate descriptions
            self.logger.info("Generating dataset descriptions...")
            response = self.agent.print_response(description_prompt, stream=True)
            
            # For now, create placeholder descriptions that will be replaced by the agent's output
            descriptions = {
                "business_description": "Business description generated by the agent.",
                "technical_description": "Technical description generated by the agent."
            }
            
            return descriptions
            
        except Exception as e:
            self.logger.error(f"Error generating dataset description: {e}")
            raise
            
    def analyze_domain_specific_patterns(self):
        """
        Analyze domain-specific patterns and insights based on the dataset.
        Returns:
            dict: A dictionary containing domain-specific insights.
        """
        try:
            # Get dataset description
            descriptions = self.generate_dataset_description()
            
            # Identify column types for domain-specific analysis
            schema = self.df.schema
            numeric_columns = [field.name for field in schema if isinstance(field.dataType, (IntegerType, DoubleType))]
            date_columns = [field.name for field in schema if isinstance(field.dataType, (DateType, TimestampType))]
            categorical_columns = [field.name for field in schema if isinstance(field.dataType, StringType) and 
                                self.df.select(field.name).distinct().count() < 50]  # Limit to manageable categories
            
            # Prepare analysis data
            analysis_data = {
                "schema": str(schema),
                "business_description": descriptions["business_description"],
                "technical_description": descriptions["technical_description"],
                "numeric_columns": numeric_columns,
                "date_columns": date_columns,
                "categorical_columns": categorical_columns,
                "total_records": self.df.count(),
                "column_correlations": {},
                "time_patterns": {},
                "category_distributions": {}
            }
            
            # Calculate correlations between numeric columns
            if len(numeric_columns) >= 2:
                for i in range(len(numeric_columns)):
                    for j in range(i+1, len(numeric_columns)):
                        col1 = numeric_columns[i]
                        col2 = numeric_columns[j]
                        correlation = self.df.stat.corr(col1, col2)
                        if correlation is not None:
                            if col1 not in analysis_data["column_correlations"]:
                                analysis_data["column_correlations"][col1] = {}
                            analysis_data["column_correlations"][col1][col2] = correlation
            
            # Analyze temporal patterns if date columns exist
            for date_col in date_columns:
                # Check if we can extract hour
                try:
                    hourly_data = self.df.withColumn("hour", hour(col(date_col)))
                    hour_counts = hourly_data.groupBy("hour").count().orderBy("hour").collect()
                    analysis_data["time_patterns"][f"{date_col}_hourly"] = hour_counts
                except:
                    pass
                
                # Check if we can extract day of week
                try:
                    daily_data = self.df.withColumn("dayofweek", dayofweek(col(date_col)))
                    day_counts = daily_data.groupBy("dayofweek").count().orderBy("dayofweek").collect()
                    analysis_data["time_patterns"][f"{date_col}_daily"] = day_counts
                except:
                    pass
                
                # Check if we can extract month
                try:
                    monthly_data = self.df.withColumn("month", month(col(date_col)))
                    month_counts = monthly_data.groupBy("month").count().orderBy("month").collect()
                    analysis_data["time_patterns"][f"{date_col}_monthly"] = month_counts
                except:
                    pass
            
            # Analyze category distributions
            for cat_col in categorical_columns:
                category_counts = self.df.groupBy(cat_col).count().orderBy(col("count").desc()).collect()
                analysis_data["category_distributions"][cat_col] = category_counts
            
            # Generate cross-tabulations for categorical columns
            if len(categorical_columns) >= 2:
                for i in range(len(categorical_columns)):
                    for j in range(i+1, len(categorical_columns)):
                        col1 = categorical_columns[i]
                        col2 = categorical_columns[j]
                        if self.df.select(col1).distinct().count() * self.df.select(col2).distinct().count() < 100:  # Limit size
                            crosstab = self.df.crosstab(col1, col2).collect()
                            if "cross_tabulations" not in analysis_data:
                                analysis_data["cross_tabulations"] = {}
                            analysis_data["cross_tabulations"][f"{col1}_{col2}"] = crosstab
            
            # Generate the agent prompt for domain-specific insights
            insights_prompt = f"""
            Based on the following analysis of the dataset, provide domain-specific insights:
            
            Business Context:
            {analysis_data['business_description']}
            
            Technical Description:
            {analysis_data['technical_description']}
            
            Numeric Columns: {analysis_data['numeric_columns']}
            Date Columns: {analysis_data['date_columns']}
            Categorical Columns: {analysis_data['categorical_columns']}
            
            Correlations between numeric columns:
            {analysis_data['column_correlations']}
            
            Time patterns:
            {analysis_data['time_patterns']}
            
            Category distributions:
            {analysis_data['category_distributions']}
            
            Please provide:
            
            1. Domain-specific insights (minimum 5):
               - Key patterns and trends in the data
               - Business implications of these patterns
               - Unusual or unexpected findings
               - Time-based patterns (if applicable)
               - Relationships between different variables
            
            2. Business recommendations (minimum 3):
               - Actions that could be taken based on these insights
               - Opportunities for improvement or optimization
               - Areas requiring further investigation
            
            3. PySpark code to:
               - Clean and transform the data
               - Perform more advanced analysis based on the domain
               - Generate relevant visualizations
            
            Make sure all insights are specific to this dataset and its domain, not generic data analysis recommendations.
            """
            
            # Use the agent to generate insights
            self.logger.info("Generating domain-specific insights...")
            self.agent.print_response(insights_prompt, stream=True)
            
            return analysis_data
            
        except Exception as e:
            self.logger.error(f"Error analyzing domain-specific patterns: {e}")
            raise

    def generate_visualizations(self):
        """
        Generate domain-appropriate visualizations for the dataset.
        Returns:
            dict: A dictionary containing visualization code and descriptions.
        """
        try:
            # Identify column types for visualizations
            schema = self.df.schema
            numeric_columns = [field.name for field in schema if isinstance(field.dataType, (IntegerType, DoubleType))]
            date_columns = [field.name for field in schema if isinstance(field.dataType, (DateType, TimestampType))]
            categorical_columns = [field.name for field in schema if isinstance(field.dataType, StringType) and 
                                self.df.select(field.name).distinct().count() < 50]
            
            # Generate visualization prompt
            viz_prompt = f"""
            Generate PySpark code to create the following visualizations for this dataset:
            
            Schema:
            {schema}
            
            Numeric Columns: {numeric_columns}
            Date Columns: {date_columns}
            Categorical Columns: {categorical_columns}
            
            Please provide code for:
            
            1. Distribution plots for key numeric columns
            2. Time series analysis plots (if date columns exist)
            3. Correlation heatmap for numeric columns
            4. Bar charts for categorical distributions
            5. Box plots comparing numeric variables across categories
            6. Domain-specific custom visualizations based on the dataset type
            
            Explain why each visualization is relevant to this specific dataset and what business insights it can provide.
            The visualization code should use PySpark, pandas, matplotlib, and/or seaborn.
            """
            
            # Use the agent to generate visualization code
            self.logger.info("Generating visualization code...")
            self.agent.print_response(viz_prompt, stream=True)
            
            # For now, return placeholder visualization code
            return {
                "visualization_code": "Visualization code generated by the agent.",
                "visualization_descriptions": "Descriptions of visualizations generated by the agent."
            }
            
        except Exception as e:
            self.logger.error(f"Error generating visualizations: {e}")
            raise

    def execute_custom_analysis(self, analysis_code):
        """
        Execute custom PySpark analysis code provided by the agent.
        Args:
            analysis_code (str): PySpark code snippet for advanced analysis.
        Returns:
            The result of the analysis execution.
        """
        try:
            self.logger.info("Executing custom analysis code...")
            # Create a local dictionary with necessary objects
            local_dict = {
                "spark": self.spark,
                "df": self.df,
                "pd": pd,
                "plt": plt,
                "sns": sns,
                "col": col
            }
            
            # Execute the code in the context with the local dictionary
            exec(analysis_code, globals(), local_dict)
            
            self.logger.info("Custom analysis executed successfully.")
            return local_dict.get("result", "Analysis executed successfully but no explicit result was returned.")
            
        except Exception as e:
            self.logger.error(f"Error executing custom analysis: {e}")
            raise

    def generate_comprehensive_report(self):
        """
        Generate a comprehensive report with all analyses and insights.
        Returns:
            dict: A dictionary containing the full report.
        """
        try:
            # Generate dataset descriptions
            descriptions = self.generate_dataset_description()
            
            # Analyze domain-specific patterns
            insights = self.analyze_domain_specific_patterns()
            
            # Generate visualizations
            visualizations = self.generate_visualizations()
            
            # Compile the comprehensive report
            report = {
                "dataset_name": self.table_name,
                "business_description": descriptions["business_description"],
                "technical_description": descriptions["technical_description"],
                "data_quality": {
                    "total_records": self.df.count(),
                    "null_counts": {col_name: self.df.filter(col(col_name).isNull()).count() for col_name in self.df.columns},
                    "duplicate_records": self.df.count() - self.df.dropDuplicates().count()
                },
                "domain_insights": insights,
                "visualizations": visualizations,
                "recommendations": "Recommendations generated by the agent based on all analyses."
            }
            
            # Generate report prompt
            report_prompt = f"""
            Based on all the analyses performed, create a comprehensive report with the following sections:
            
            1. Executive Summary
            2. Dataset Overview
               - {report['business_description']}
               - {report['technical_description']}
            3. Data Quality Assessment
               - Total Records: {report['data_quality']['total_records']}
               - Null Values: {report['data_quality']['null_counts']}
               - Duplicate Records: {report['data_quality']['duplicate_records']}
            4. Key Insights
               - {report['domain_insights']}
            5. Visualizations
               - {report['visualizations']}
            6. Recommendations
            7. Next Steps
            
            Make sure all insights and recommendations are specific to this dataset and its domain.
            """
            
            # Use the agent to generate the final report
            self.logger.info("Generating comprehensive report...")
            self.agent.print_response(report_prompt, stream=True)
            
            return report
            
        except Exception as e:
            self.logger.error(f"Error generating comprehensive report: {e}")
            raise

# Usage
if __name__ == "__main__":
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("EnhancedDataAnalysis") \
        .getOrCreate()
    
    # Initialize the agent
    table_name = "LH_NYC_taxi.green_tripdata_2017"  # Replace with your table name
    data_analyzer = EnhancedDataAnalysisAgent(spark, table_name)
    
    # Generate comprehensive report
    try:
        report = data_analyzer.generate_comprehensive_report()
        print("Comprehensive Data Analysis Report Generated Successfully")
    except Exception as e:
        print(f"Error during analysis: {e}")
    
    # Stop Spark session
    spark.stop()

▰▰▰▰▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃             Based on all the analyses performed, create a comprehensive report with the following sections:     ┃
┃                                                                                                                 ┃
┃             1. Executive Summary                                                                                ┃
┃             2. Dataset Overview                                                                                 ┃
┃                - Business description generated by the agent.                                                   ┃
┃                - Technical description generated by the agent.                                                  ┃
┃             3. Data Quality Assessment                                                                          ┃
┃                - Total Records: 1000                                                                            ┃
┃                - Null Values: {'VendorID': 0, 'lpep_pickup_datetime': 0, 'lpep_dropoff_datetime': 0,            ┃
┃ 'store_and_fwd_flag': 0, 'RatecodeID': 0, 'PULocationID': 0, 'DOLocationID': 0, 'passenger_count': 0,           ┃
┃ 'trip_distance': 0, 'fare_amount': 0, 'extra': 0, 'mta_tax': 0, 'tip_amount': 0, 'tolls_amount': 0,             ┃
┃ 'ehail_fee': 1000, 'improvement_surcharge': 0, 'total_amount': 0, 'payment_type': 0, 'trip_type': 0,            ┃
┃ 'congestion_surcharge': 1000, 'source_file': 0}                                                                 ┃
┃                - Duplicate Records: 0                                                                           ┃
┃             4. Key Insights                                                                                     ┃
┃                - {'schema': "StructType([StructField('VendorID', LongType(), True),                             ┃
┃ StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime',                ┃
┃ TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID',        ┃
┃ LongType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(),       ┃
┃ True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True),      ┃
┃ StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True),                       ┃
┃ StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True),                      ┃
┃ StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True),                  ┃
┃ StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True),      ┃
┃ StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True),                      ┃
┃ StructField('congestion_surcharge', DoubleType(), True), StructField('source_file', StringType(), True)])",     ┃
┃ 'business_description': 'Business description generated by the agent.', 'technical_description': 'Technical     ┃
┃ description generated by the agent.', 'numeric_columns': ['trip_distance', 'fare_amount', 'extra', 'mta_tax',   ┃
┃ 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'],    ┃
┃ 'date_columns': ['lpep_pickup_datetime', 'lpep_dropoff_datetime'], 'categorical_columns':                       ┃
┃ ['store_and_fwd_flag', 'source_file'], 'total_records': 1000, 'column_correlations': {'trip_distance':          ┃
┃ {'fare_amount': 0.9101694814956953

Comprehensive Data Analysis Report Generated Successfully
